In [2]:
#Classification Metrics

In [8]:
import pandas as pd

# Define the URL for the dataset
data_web_address = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# Define the column names for the dataset
column_names = ['pregnancy_x', 
                'plasma_con', 
                'blood_pressure', 
                'skin_mm', 
                'insulin', 
                'bmi', 
                'pedigree_func', 
                'age', 
                'target']

# Load the dataset into a pandas DataFrame
all_data = pd.read_csv(data_web_address, names=column_names)

# Display the first few rows of the dataset to ensure it is loaded correctly
print(all_data.head())



   pregnancy_x  plasma_con  blood_pressure  skin_mm  insulin   bmi  \
0            6         148              72       35        0  33.6   
1            1          85              66       29        0  26.6   
2            8         183              64        0        0  23.3   
3            1          89              66       23       94  28.1   
4            0         137              40       35      168  43.1   

   pedigree_func  age  target  
0          0.627   50       1  
1          0.351   31       0  
2          0.672   32       1  
3          0.167   21       0  
4          2.288   33       1  


In [10]:
import numpy as np
import pandas as pd

X = all_data[feature_names]
y = all_data['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7,stratify=y)

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV

knn_clf = KNeighborsClassifier()

param_dist = {'n_neighbors': list(range(3,20,1))}

rs = RandomizedSearchCV(knn_clf,param_dist,cv=10,n_iter=17)
rs.fit(X_train, y_train)

RandomizedSearchCV(cv=10, estimator=KNeighborsClassifier(), n_iter=17,
                   param_distributions={'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10,
                                                        11, 12, 13, 14, 15, 16,
                                                        17, 18, 19]})

In [14]:
rs.best_score_

0.7539397144368059

In [16]:
y_pred = rs.predict(X_test)

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[84, 16],
       [27, 27]])

In [18]:
from sklearn.metrics import recall_score

recall_score(y_test, y_pred)

0.5

In [20]:
from sklearn.metrics import make_scorer

recall_scorer = make_scorer(recall_score, greater_is_better=True)

In [22]:
from sklearn.metrics import make_scorer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV

knn_clf = KNeighborsClassifier()

param_dist = {'n_neighbors': list(range(3,20,1))}

rs = RandomizedSearchCV(knn_clf,param_dist,cv=10,n_iter=17,scoring=recall_scorer)

rs.fit(X_train, y_train)

RandomizedSearchCV(cv=10, estimator=KNeighborsClassifier(), n_iter=17,
                   param_distributions={'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10,
                                                        11, 12, 13, 14, 15, 16,
                                                        17, 18, 19]},
                   scoring=make_scorer(recall_score, response_method='predict'))

In [24]:
rs.best_score_

0.5647186147186147

In [26]:
y_pred = rs.predict(X_test)
recall_score(y_test,y_pred)

0.5

In [28]:
from sklearn.metrics import roc_auc_score

rs = RandomizedSearchCV(knn_clf,param_dist,cv=10,n_iter=17,scoring=make_scorer(roc_auc_score,greater_is_better=True))
rs.fit(X_train, y_train)
rs.best_score_

0.7098593073593072

In [30]:
costs_array = confusion_matrix(y_test, y_pred) * np.array([[1,2],
                                            [100,20]])
costs_array

array([[  84,   32],
       [2700,  540]])

In [32]:
costs_array.sum()

3356

In [34]:
def costs_total(y_test, y_pred):
    return (confusion_matrix(y_test, y_pred) * np.array([[1,2],[100,20]])).sum()

costs_scorer = make_scorer(costs_total, greater_is_better=False)

rs = RandomizedSearchCV(knn_clf,param_dist,cv=10,n_iter=17,scoring=costs_scorer)

rs.fit(X_train, y_train)
rs.best_score_

-1217.8

In [36]:
costs_total(y_test,rs.predict(X_test)) 

3356